# Tuning rate, w 

In [1]:
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from bokeh.layouts import column, row
from bokeh.models import Range1d
output_notebook()

import numpy as np

from fakespikes.util import kappa
from fakespikes.util import fano

from resistingrhythm.neurons import HHH
from resistingrhythm.util import poisson_impulse
from resistingrhythm.util import poisson_oscillation
from resistingrhythm.util import poisson_drift_rate
from resistingrhythm.util import create_times

Loading BokehJS ...

/Users/type/anaconda/envs/py2/lib/python2.7/site-packages/brian2/core/variables.py:174: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  return np.issubdtype(np.bool, self.dtype)


# Set parameters

In [2]:
# ------------------------------------------------
# User params:
time = 4.0
tau_h = 1
percent_change = 0.17

# Spikes
N_stim = 1000
stim_rate = 20
stim_onset = 0.2
stim_offset = time

N_osc = 100
f = 10
osc_rate = 10
osc_onset = 0.2
n_cycles = int(time * f * 2)  # Inf. sustained....

# HHH
w_stim = 3e-6
w_osc = w_stim / 10

bias_in=0e-9  # 138e-9 generates robust spiking

Ca_equillibrium = 0.0116602728499
homeostasis = True

# ------------------------------------------------
# Fixed
time_step = 1e-5
N = 1
seed_stim = None

# Create stim and osc spikes

In [10]:
ns_stim, ts_stim, times, xs = poisson_drift_rate(
    time,
    stim_onset,
    tau=.5,
    sigma=0.01,
    rate=6.0,
    n=N_stim,
    dt=1e-4,
    min_rate=0.5
)

# Generate the rate 
# ns_osc, ts_osc = poisson_oscillation(
#     time, 
#     osc_onset,
#     n_cycles,
#     osc_rate,
#     f,
#     phi=0,
#     n=N_osc,
#     dt=1e-4,
#     seed=seed_stim
# )

In [11]:
p = figure(plot_width=400, plot_height=200)
for n in range(N):
    p.circle(x=times, y=xs, color="grey", alpha=0.5, size=0.01)
    p.line(x=times, y=xs, color="grey", alpha=0.5)
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'Stim rate (Hz)'
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.x_range = Range1d(0, time)
show(p)

In [12]:
p = figure(plot_width=400, plot_height=200)
p.circle(ts_stim, ns_stim, color="black", size=0.1)
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'N'
p.x_range = Range1d(0, time)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

In [ ]:
p = figure(plot_width=400, plot_height=200)
p.circle(ts_osc, ns_osc, color="black", size=0.1)
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'N'
p.x_range = Range1d(0, time)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

# Run HHH, stim only

In [ ]:
results0 = HHH(
    time,
    ns_stim,
    ts_stim,
    np.asarray([]), # osc
    np.asarray([]), # osc
    N=N,
    Ca=Ca_equillibrium,
    Ca_target=Ca_equillibrium + (Ca_equillibrium * percent_change),
    w_in=w_stim,
    w_osc=w_osc,
    bias_in=bias_in,
    sigma=0,
    tau_h=tau_h,
    time_step=time_step,
    homeostasis=True
)

## Analyze

In [ ]:
p = figure(plot_width=400, plot_height=200)
for n in range(N):
    p.line(x=results0['times'], y=results0['v_m'][n, :], 
           color="grey", alpha=0.5)
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'V_m (volts)'
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.x_range = Range1d(0, time)
# p.y_range = Range1d(-70e-3, -60e-3)
show(p)

In [ ]:
p = figure(plot_width=400, plot_height=200)
p.circle(results0['ts'], results0['ns'], color="black", size=0.1)
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'N'
p.x_range = Range1d(0, time)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

In [ ]:
print(">>> Avg. firing rate {}".format(results0['ns'].size / time))

In [ ]:
p = figure(plot_width=400, plot_height=200)
for n in range(N):
    p.line(x=results0['times'], 
           y=results0['calcium'][n, :], 
           color="grey", alpha=0.5)
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'Ca (moles)'
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.x_range = Range1d(0, time)
show(p)

# Run with stim, and osc mod

In [ ]:
results1 = HHH(
    time,
    ns_stim,
    ts_stim,
    ns_osc,
    ts_osc,
    N=N,
    Ca=Ca_equillibrium,
    Ca_target=Ca_equillibrium + (Ca_equillibrium * percent_change),
    w_in=w_stim,
    w_osc=w_osc,
    bias_in=bias_in,
    sigma=0,
    tau_h=tau_h,
    time_step=time_step,
    homeostasis=True
)

In [ ]:
p = figure(plot_width=400, plot_height=200)
for n in range(N):
    p.line(x=results1['times'], y=results1['v_m'][n, :], 
           color="grey", alpha=0.5)
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'V_m (volts)'
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.x_range = Range1d(0, time)
# p.y_range = Range1d(-70e-3, -60e-3)
show(p)

In [ ]:
p = figure(plot_width=400, plot_height=200)
p.circle(results1['ts'], results1['ns'], color="black", size=0.1)
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'N'
p.x_range = Range1d(0, time)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

In [ ]:
print(">>> Avg. firing rate {}".format(results1['ns'].size / time))

In [ ]:
p = figure(plot_width=400, plot_height=200)
for n in range(N):
    p.line(x=results1['times'], 
           y=results1['calcium'][n, :], 
           color="grey", alpha=0.5)
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'Ca (moles)'
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.x_range = Range1d(0, time)
show(p)